In [1]:
import pandas as pd
man_df = pd.read_csv('data/caesar_all_fr.csv', encoding='utf-8', sep=';')
man_df=man_df.sample(frac=1)
caesar_female = man_df[man_df['sex'] == 0]
caesar_female=caesar_female.loc[:, (caesar_female != 0).any()]

In [2]:
# Drop rows which include NaN
caesar_female = caesar_female.dropna()

In [3]:
caesar_female.isna().sum()

taille                                                0
age                                                   0
weight                                                0
tour_de_cheville                                      0
longueur_du_bras                                      0
tour_de_poitrine                                      0
hauteur_de_poitrine                                   0
hauteur_d_entrejambe                                  0
hauteur_des_hanches                                   0
hauteur_des_genoux                                    0
tour_du_cou                                           0
largeur_d_epaule                                      0
hauteur_des_epaules                                   0
largeur_des_epaules_a_l_horizontales                  0
longueur_de_la_colonne_vertebrale_jusqu_au_poignet    0
tour_de_cuisse                                        0
tour_de_taille                                        0
hauteur_de_la_taille                            

In [4]:
caesar_female["tour_de_hanches_ratio"] = caesar_female["tour_de_hanches"] / caesar_female["taille"]
caesar_female["tour_de_taille_ratio"] = caesar_female["tour_de_taille"] / caesar_female["taille"]

In [5]:
def classifier_morphologie(row):
    # Extraction des mesures
    #tour_ventre = row['tour_de_ventre_maximal']
    forme_ventre = row['tour_de_taille_ratio']
    forme_bassin = row['tour_de_hanches_ratio']
    
    
    # --- Catégorie VENTRE (basée sur le ratio ventre/taille) ---
    if forme_ventre < 0.431483:
        ventre = "plat"
    elif 0.431483 <= forme_ventre<0.520135:
        ventre = "moyen"
    else:
        ventre = "rond"
    
    # --- Catégorie BASSIN (basée sur le tour de hancehs ) ---
    if forme_bassin < 0.588740:
        bassin = "etroit"
    elif 0.588740 <=forme_bassin <0.662996 :
        bassin = "moyen"
    else:
        bassin = "large"
    

    
    return pd.Series([ventre, bassin], 
                     index=['categorie_ventre', 'categorie_bassin'])
    
# Remplace `df` par le nom de ton DataFrame
caesar_female[['categorie_ventre', 'categorie_bassin']] = caesar_female.apply(classifier_morphologie, axis=1)

In [6]:
caesar_female["bonnet"]=caesar_female["tour_de_poitrine"]-caesar_female["tour_de_sous_poitrine"]

In [7]:
def calculer_bonnet(diff):
    if pd.isna(diff):
        return None
    elif diff < 13:
        return 'AA'
    elif diff < 15:
        return 'A'
    elif diff < 17:
        return 'B'
    elif diff < 19:
        return 'C'
    elif diff < 21:
        return 'D'
    elif diff < 23:
        return 'E'
    elif diff < 25:
        return 'F'
    elif diff < 27:
        return 'G'
    elif diff < 29:
        return 'H'
    elif diff < 31:
        return 'I'
    elif diff < 33:
        return 'J'
    else:
        return 'K'

caesar_female['bonnet_lettre'] = caesar_female['bonnet'].map(calculer_bonnet)

def arrondir_taille(val):
    if pd.isna(val):
        return None
    return int(5 * round(val / 5))

caesar_female['taille_soutien_gorge'] = caesar_female['taille_de_poitrine'].apply(arrondir_taille)

caesar_female.head(10)

,taille,age,weight,tour_de_cheville,longueur_du_bras,tour_de_poitrine,hauteur_de_poitrine,hauteur_d_entrejambe,hauteur_des_hanches,hauteur_des_genoux,...,taille_de_poitrine,bonnet_de_soutien_gorge,tour_de_hanches,tour_de_hanches_ratio,tour_de_taille_ratio,categorie_ventre,categorie_bassin,bonnet,bonnet_lettre,taille_soutien_gorge
68,168.9100,54.0,73.708700,23.0124,59.8932,97.2058,121.7930,76.9112,92.6084,46.2026,...,91.44,1.0,106.5022,0.630526,0.533985,rond,moyen,13.1064,A,90
2255,165.9890,46.0,108.862080,26.1874,61.4934,127.5080,115.7986,74.1934,89.0016,44.3992,...,106.68,5.0,135.7122,0.817598,0.668707,rond,large,22.6060,E,105
382,159.2072,43.0,71.667536,24.3078,54.7878,102.3112,117.1956,70.9930,77.6986,43.7896,...,91.44,3.0,106.5022,0.668953,0.516911,moyen,large,18.7198,C,90
3296,170.9928,41.0,56.472204,23.9014,62.8904,84.3026,123.3932,78.5876,83.3120,45.9994,...,86.36,1.0,91.9988,0.538027,0.388295,plat,etroit,8.2042,AA,85
4210,154.4066,44.7,49.500495,22.4028,57.9882,84.5058,111.5060,70.1040,72.9996,43.3578,...,81.28,2.0,97.5106,0.631518,0.476065,moyen,moyen,15.3162,B,80
488,171.2976,47.0,75.296272,25.4000,59.5122,97.2058,126.7968,79.2988,83.1088,47.1932,...,101.60,1.0,110.5916,0.645611,0.457147,moyen,moyen,14.0970,A,100
2288,175.9966,49.0,73.935496,26.5938,60.9092,94.8944,129.9972,82.4992,87.0966,48.3108,...,91.44,2.0,106.5022,0.605138,0.428922,plat,moyen,16.1036,B,90
28,167.5892,39.0,58.740164,23.0124,59.2074,87.6046,123.5964,79.0956,81.0006,46.7995,...,91.44,3.0,97.5106,0.581843,0.423613,plat,etroit,15.3924,B,90
3082,164.3888,23.0,47.627160,22.8092,57.6072,77.8002,119.4054,77.6986,78.8924,44.7040,...,81.28,2.0,88.3920,0.537701,0.371755,plat,etroit,10.8966,AA,80
508,165.7096,21.0,77.564232,23.7998,61.8998,96.3930,121.5898,79.5020,81.7880,46.4058,...,91.44,3.0,113.9952,0.687922,0.483906,moyen,large,19.4056,D,90


In [8]:
# Dictionnaire de correspondance bonnet → rang
bonnet_scale = {
    'AA': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4,
    'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11
}

# Appliquer les rangs
caesar_female['bonnet_rang'] = caesar_female['bonnet_lettre'].map(bonnet_scale)
caesar_female.head(10)

,taille,age,weight,tour_de_cheville,longueur_du_bras,tour_de_poitrine,hauteur_de_poitrine,hauteur_d_entrejambe,hauteur_des_hanches,hauteur_des_genoux,...,bonnet_de_soutien_gorge,tour_de_hanches,tour_de_hanches_ratio,tour_de_taille_ratio,categorie_ventre,categorie_bassin,bonnet,bonnet_lettre,taille_soutien_gorge,bonnet_rang
68,168.9100,54.0,73.708700,23.0124,59.8932,97.2058,121.7930,76.9112,92.6084,46.2026,...,1.0,106.5022,0.630526,0.533985,rond,moyen,13.1064,A,90,1
2255,165.9890,46.0,108.862080,26.1874,61.4934,127.5080,115.7986,74.1934,89.0016,44.3992,...,5.0,135.7122,0.817598,0.668707,rond,large,22.6060,E,105,5
382,159.2072,43.0,71.667536,24.3078,54.7878,102.3112,117.1956,70.9930,77.6986,43.7896,...,3.0,106.5022,0.668953,0.516911,moyen,large,18.7198,C,90,3
3296,170.9928,41.0,56.472204,23.9014,62.8904,84.3026,123.3932,78.5876,83.3120,45.9994,...,1.0,91.9988,0.538027,0.388295,plat,etroit,8.2042,AA,85,0
4210,154.4066,44.7,49.500495,22.4028,57.9882,84.5058,111.5060,70.1040,72.9996,43.3578,...,2.0,97.5106,0.631518,0.476065,moyen,moyen,15.3162,B,80,2
488,171.2976,47.0,75.296272,25.4000,59.5122,97.2058,126.7968,79.2988,83.1088,47.1932,...,1.0,110.5916,0.645611,0.457147,moyen,moyen,14.0970,A,100,1
2288,175.9966,49.0,73.935496,26.5938,60.9092,94.8944,129.9972,82.4992,87.0966,48.3108,...,2.0,106.5022,0.605138,0.428922,plat,moyen,16.1036,B,90,2
28,167.5892,39.0,58.740164,23.0124,59.2074,87.6046,123.5964,79.0956,81.0006,46.7995,...,3.0,97.5106,0.581843,0.423613,plat,etroit,15.3924,B,90,2
3082,164.3888,23.0,47.627160,22.8092,57.6072,77.8002,119.4054,77.6986,78.8924,44.7040,...,2.0,88.3920,0.537701,0.371755,plat,etroit,10.8966,AA,80,0
508,165.7096,21.0,77.564232,23.7998,61.8998,96.3930,121.5898,79.5020,81.7880,46.4058,...,3.0,113.9952,0.687922,0.483906,moyen,large,19.4056,D,90,4


In [9]:
# 1. Wrapper
class MultiTPOTWrapper:
    def __init__(self, models):
        self.models = models

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        import numpy as np
        predictions = [m.predict(X) for m in self.models]
        return np.vstack(predictions).T

In [10]:
# 2. Pré-traitement
from sklearn.preprocessing import RobustScaler, MinMaxScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib


X_train=caesar_female[['taille', 'age', 'weight','taille_soutien_gorge',
                       'bonnet_rang','categorie_ventre', 'categorie_bassin']]

# Colonnes numériques et catégorielles
cols_num = ["taille","age","weight","taille_soutien_gorge"]
col_bon=["bonnet_rang"]
cols_cat = ["categorie_ventre", "categorie_bassin"]

# Pipeline pour les colonnes numériques : imputation par médiane + scaling
pipeline_num = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

pipeline_bon = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", MinMaxScaler())
])

# Pipeline pour les colonnes catégorielles : imputation par la valeur la plus fréquente + one-hot
pipeline_cat = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline globale de prétraitement
preprocessor = ColumnTransformer(transformers=[
    ("num", pipeline_num, cols_num),
    ("bon", pipeline_bon, col_bon),
    ("cat", pipeline_cat, cols_cat)

])

# Fit sur X_train (tu dois définir X_train comme DataFrame brut)
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['taille', 'age', 'weight',
                                  'taille_soutien_gorge']),
                                ('bon',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', MinMaxScaler())]),
                                 ['bonnet_rang']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['categorie_ventre', 'categorie_bassin'])])

In [38]:
caesar_female['taille_soutien_gorge'].value_counts()

taille_soutien_gorge
85     744
90     610
95     286
80     190
100    109
105     60
110     43
115     13
Name: count, dtype: int64

In [11]:
import joblib
import os

# 📁 Répertoire contenant les fichiers .pkl
dossier_modeles = "./modeles_female_tpot_complet"  # modifie ce chemin si nécessaire

# 📌 Chargement de tous les modèles
modeles = {}
for nom_fichier in os.listdir(dossier_modeles):
    if nom_fichier.endswith(".pkl"):
        nom_cible = nom_fichier.replace("tpot_", "").replace(".pkl", "")
        chemin_complet = os.path.join(dossier_modeles, nom_fichier)
        modeles[nom_cible] = joblib.load(chemin_complet)

print(f"✅ {len(modeles)} modèles chargés :", list(modeles.keys()))

✅ 20 modèles chargés : ['hauteur_des_epaules', 'hauteur_des_genoux', 'hauteur_des_hanches', 'hauteur_de_la_taille', 'hauteur_de_poitrine', 'hauteur_d_entrejambe', 'largeur_des_epaules_a_l_horizontales', 'largeur_de_mamelon_a_mamelon', 'largeur_d_epaule', 'longueur_de_la_colonne_vertebrale_jusqu_au_poignet', 'longueur_du_bras', 'longueur_d_avant_bras', 'taille_de_poitrine', 'tour_de_cheville', 'tour_de_cuisse', 'tour_de_hanches', 'tour_de_poitrine', 'tour_de_sous_poitrine', 'tour_de_taille', 'tour_du_cou']


In [12]:
import os

dossier_modeles = "./modeles_female_tpot_complet"  # adapte ce chemin
noms_fichiers = [f for f in os.listdir(dossier_modeles) if f.endswith(".pkl")]

# Dossier de sauvegarde des pipelines complets
dossier_output = "pipelines_female_complets"
os.makedirs(dossier_output, exist_ok=True)

for nom_fichier in noms_fichiers:
    chemin_modele = os.path.join(dossier_modeles, nom_fichier)
    
    # Charger le modèle TPOT
    tpot_model = joblib.load(chemin_modele)
    
    # Créer le pipeline complet
    full_pipeline = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", tpot_model)
    ])
    
    # Sauvegarder le pipeline complet
    nom_fichier_pipeline = f"pipeline_{nom_fichier}"
    chemin_output = os.path.join(dossier_output, nom_fichier_pipeline)
    joblib.dump(full_pipeline, chemin_output)

    print(f"✔ Pipeline sauvegardé : {chemin_output}")

✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_hauteur_des_epaules.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_hauteur_des_genoux.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_hauteur_des_hanches.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_hauteur_de_la_taille.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_hauteur_de_poitrine.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_hauteur_d_entrejambe.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_largeur_des_epaules_a_l_horizontales.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_largeur_de_mamelon_a_mamelon.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_largeur_d_epaule.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_longueur_de_la_colonne_vertebrale_jusqu_au_poignet.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_l

In [13]:
# 3. Chargement modèles
import joblib
import os

dossier_pipelines = "pipelines_female_complets"
models = []
for filename in sorted(os.listdir(dossier_pipelines)):
    if filename.endswith(".pkl"):
        path = os.path.join(dossier_pipelines, filename)
        models.append(joblib.load(path))
        print(f"✅ Modèle chargé : {filename}")

✅ Modèle chargé : pipeline_tpot_hauteur_d_entrejambe.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_de_la_taille.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_de_poitrine.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_des_epaules.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_des_genoux.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_des_hanches.pkl
✅ Modèle chargé : pipeline_tpot_largeur_d_epaule.pkl
✅ Modèle chargé : pipeline_tpot_largeur_de_mamelon_a_mamelon.pkl
✅ Modèle chargé : pipeline_tpot_largeur_des_epaules_a_l_horizontales.pkl
✅ Modèle chargé : pipeline_tpot_longueur_d_avant_bras.pkl
✅ Modèle chargé : pipeline_tpot_longueur_de_la_colonne_vertebrale_jusqu_au_poignet.pkl
✅ Modèle chargé : pipeline_tpot_longueur_du_bras.pkl
✅ Modèle chargé : pipeline_tpot_taille_de_poitrine.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_cheville.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_cuisse.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_hanches.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_poitrine.pkl
✅ Modèle c

In [14]:
# 4. Pipeline complet
full_pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("multi_model", MultiTPOTWrapper(models))
])

In [15]:
# 5. Sauvegarde
joblib.dump(full_pipeline, "full_pipeline_female.pkl")

# 6. Prédiction
pipeline_loaded = joblib.load("full_pipeline_female.pkl")

In [27]:
# ✅ Créer un wrapper à partir des pipelines chargés
wrapper = MultiTPOTWrapper(models)
new_data = pd.DataFrame([{
    "taille": 168,
    "age": 28,
    "weight": 75,
    "categorie_ventre": "plat",
    "categorie_bassin": "large",
    "taille_soutien_gorge": 123,
    "bonnet_rang":5
    
}])
# ✅ Prédictions
preds = wrapper.predict(new_data)


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-datas

In [28]:
# Suppose que tu as déjà ton pipeline chargé
# pipeline_loaded est de type Pipeline(steps=[("preprocess", ...), ("multi_model", ...)])

# Accès au préprocesseur uniquement
preprocessor = pipeline_loaded.named_steps["preprocess"]

# Applique le transformateur au new_data brut
X = preprocessor.transform(new_data)

# Si tu veux voir sous forme de DataFrame
import pandas as pd

# Récupère les noms des colonnes après transformation
# Cela fonctionne uniquement si le ColumnTransformer est simple
try:
    feature_names = preprocessor.get_feature_names_out()
    X_df = pd.DataFrame(X, columns=feature_names)
except:
    # fallback si les noms ne sont pas récupérables
    X_df = pd.DataFrame(X)

# Affiche les premières lignes
X_df.head()

,num__taille,num__age,num__weight,num__taille_soutien_gorge,bon__bonnet_rang,cat__categorie_ventre_moyen,cat__categorie_ventre_plat,cat__categorie_ventre_rond,cat__categorie_bassin_etroit,cat__categorie_bassin_large,cat__categorie_bassin_moyen
0,0.372181,-0.47619,0.548915,6.6,0.454545,0.0,1.0,0.0,0.0,1.0,0.0


In [18]:
# ✅ Extraire les noms des cibles
noms_variables = [f.replace("pipeline_tpot_", "").replace(".pkl", "") 
                  for f in sorted(os.listdir(dossier_pipelines)) if f.endswith(".pkl")]

# ✅ Associer les valeurs
predictions_dict = dict(zip(noms_variables, preds[0]))

# ✅ Affichage
print("📊 Prédictions :")
for k, v in predictions_dict.items():
    print(f" - {k} : {v:.2f}")


📊 Prédictions :
 - hauteur_d_entrejambe : 78.26
 - hauteur_de_la_taille : 101.67
 - hauteur_de_poitrine : 121.60
 - hauteur_des_epaules : 137.69
 - hauteur_des_genoux : 45.75
 - hauteur_des_hanches : 82.75
 - largeur_d_epaule : 39.61
 - largeur_de_mamelon_a_mamelon : 20.22
 - largeur_des_epaules_a_l_horizontales : 44.74
 - longueur_d_avant_bras : 24.16
 - longueur_de_la_colonne_vertebrale_jusqu_au_poignet : 53.83
 - longueur_du_bras : 59.55
 - taille_de_poitrine : 96.52
 - tour_de_cheville : 24.58
 - tour_de_cuisse : 61.90
 - tour_de_hanches : 105.88
 - tour_de_poitrine : 98.69
 - tour_de_sous_poitrine : 87.82
 - tour_de_taille : 93.83
 - tour_du_cou : 43.69


In [19]:
import pandas as pd

# Liste des noms de colonnes
colonnes = [
    "hauteur_d_entrejambe",
    "hauteur_de_la_taille",
    "hauteur_de_poitrine",
    "hauteur_des_epaules",
    "hauteur_des_genoux",
    "hauteur_des_hanches",
    "largeur_d_epaule",
    "largeur_de_mamelon_a_mamelon",
    "largeur_des_epaules_a_l_horizontales",
    "longueur_d_avant_bras",
    "longueur_de_la_colonne_vertebrale_jusqu_au_poignet",
    "longueur_du_bras",
    "taille_de_poitrine",
    "tour_de_cheville",
    "tour_de_cuisse",
    "tour_de_hanches",
    "tour_de_poitrine",
    "tour_de_sous_poitrine",
    "tour_de_taille",
    "tour_du_cou"
]

# Création d'un DataFrame vide avec seulement les colonnes
df = pd.DataFrame(columns=colonnes)

# Enregistrement dans un fichier CSV
df.to_csv("data/mesures_morphologiques_vides.csv", encoding='utf-8', sep=';', index=False)

print("✅ Fichier CSV vide avec colonnes créé : data/mesures_morphologiques_vides.csv")


✅ Fichier CSV vide avec colonnes créé : data/mesures_morphologiques_vides.csv
